# Wine composition and aromatic characteristics. Data preparation and EDA


In [11]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import re
from geopy.geocoders import Nominatim
import geopandas as gpd

## 1. Data Description

In [12]:
colors = ['#4fabd0', '#dbe9f4', '#f0d0db', '#edb9c6', '#13979c',
                 '#b3d8e8', '#f2f2f2', '#e0b0ff', '#ffabab', '#ffd8b1',
                 '#7fcdbb', '#41b6c4', '#1d91c0', '#225ea8', '#253494',
                  '#081d58', '#f7fcfd', '#fee0d2', '#fcbba1', '#fc9272']

In [13]:
wine_aroma_df = pd.read_csv('WineAroma.csv')
wine_aroma_df

,Wine,Grape sort,Year,Region,Country,Acetoine,Acetaldehyde,Methanol,1-propanol,Ethyl lactate,...,Herbs and spices,Tobacco/Smoke,Wood,Berries,Citrus,Fruits,Nuts,Coffee,Chocolate/Cacao,Flowers
0,Nobile,Rubin,2017,NaN,Bulgaria,0.0,33.76,110.03,0.0,0.000,...,0,1,0,1,0,0,0,0,1,0
1,Vidinska Gamza,Storgozia,2017,NaN,Bulgaria,0.0,24.12,141.95,0.0,0.000,...,1,1,0,1,0,0,0,0,1,0
2,Traversa,Tannat,2000,NaN,Uruguay,0.0,0.00,0.00,0.0,0.000,...,0,0,0,1,0,0,0,0,0,0
3,La Comtesse,Albarino,2015,Pontevedra,Spain,0.0,0.00,0.00,0.0,0.000,...,1,1,0,1,0,1,0,1,1,0
4,Armas de Lanzos,Albarino,2015,Pontevedra,Spain,0.0,0.00,0.00,0.0,0.000,...,1,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,Brisas del Este,Tannat,2005,Canelones,Uruguay,0.0,0.00,0.00,0.0,3.022,...,0,0,0,1,0,1,0,0,0,0
445,Bouza,Tannat,2005,Canelones,Uruguay,0.0,0.00,0.00,0.0,4.892,...,0,0,0,1,0,0,0,0,0,0
446,Salida,Tannat,2005,Canelones,Uruguay,0.0,0.00,0.00,0.0,6.256,...,1,0,0,1,0,0,0,0,0,0
447,Single Vineyard,Tannat,2005,Canelones,Uruguay,0.0,0.00,0.00,0.0,118.317,...,0,1,0,1,0,0,1,0,1,1


In [14]:
# Removing extra spaces from the columns Region, Country, Grapesort, Wine
def strip_str(str):
    try:
        return str.strip()
    except:
        return str

wine_aroma_df['Region'] = wine_aroma_df['Region'].apply(strip_str)
wine_aroma_df['Country'] = wine_aroma_df['Country'].apply(strip_str)
wine_aroma_df['Grape sort'] = wine_aroma_df['Grape sort'].apply(strip_str)
wine_aroma_df['Wine'] = wine_aroma_df['Wine'].apply(strip_str)


### Description of the dataset
This dataset was assembled by hand and contains information on 449 wines, including their chemical composition and flavor

- `Wine` - The name of the wine
- `Grape sort` - Grape variety
- `Year` - The year of wine production
- `Region` - The region where grapes grow
- `Country` - Country of wine production
- `Acetoine` - `Dodecanoic acid` - concentrations (mg/l)
- `DOI` - Link to the article
- `Descriptors` - Wine flavor
- `Herbs and spices`, `Tobacco/Smoke`, `Wood`, `Berries`, `Citrus`,
`Fruits`, `Nuts`, `Coffee`, `Chocolate/Cacao`, `Flowers` - Classification of wine flavors into 10 groups


In [ ]:
wine_aroma_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449 entries, 0 to 448
Data columns (total 66 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Wine                       446 non-null    object 
 1   Grape sort                 443 non-null    object 
 2   Year                       449 non-null    int64  
 3   Region                     412 non-null    object 
 4   Country                    444 non-null    object 
 5   Acetoine                   449 non-null    float64
 6   Acetaldehyde               449 non-null    float64
 7   Methanol                   449 non-null    float64
 8   1-propanol                 449 non-null    float64
 9   Ethyl lactate              449 non-null    float64
 10  Isobutanol                 449 non-null    float64
 11  1-butanol                  449 non-null    float64
 12  2-butanol                  449 non-null    float64
 13  2-methyl-1-butanol         449 non-null    float64

In [16]:
wine_aroma_df.describe() #Main characteristics of numeric columns

,Year,Acetoine,Acetaldehyde,Methanol,1-propanol,Ethyl lactate,Isobutanol,1-butanol,2-butanol,2-methyl-1-butanol,...,Herbs and spices,Tobacco/Smoke,Wood,Berries,Citrus,Fruits,Nuts,Coffee,Chocolate/Cacao,Flowers
count,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,...,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000
mean,2012.329621,3.013686,9.370619,10.108518,6.276215,16.704722,15.561368,0.812189,0.067253,16.465266,...,0.619154,0.104677,0.164811,0.505568,0.240535,0.674833,0.075724,0.051225,0.104677,0.336303
std,6.231619,17.931572,33.164155,27.588826,15.327420,56.265246,34.157441,2.098714,0.481374,38.685584,...,0.486137,0.306478,0.371423,0.500527,0.427885,0.468960,0.264851,0.220702,0.306478,0.472971
min,1990.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2008.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2015.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000800,0.000000,0.000000,...,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,2017.000000,0.000000,0.000000,0.000000,4.000000,6.256000,21.590000,0.640000,0.000000,8.140000,...,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
max,2020.000000,234.000000,347.000000,161.000000,182.000000,652.152000,425.000000,13.420000,8.093000,373.020000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


**Output:** The data is complete, omissions occur either in the name of the wine or in the country/region of growth. All chemical data, as well as data on the flavor of the wine, are present, so they can be used further

## 2. Analysis of descriptive data about wine
In this paragraph, we will analyze which wines are presented in the dataset, where they grew, as well as the years of their production

### 2.1. Country
Let's estimate the distribution of wines by country

In [ ]:
# Creating a pie chart
fig = px.pie(wine_aroma_df, names='Country', color_discrete_sequence = px.colors.sequential.Teal_r, labels={'Country': ''}, width=800,  height=650)

# Setting labels
fig.update_traces(textposition='inside', textinfo='label+percent', textfont_size=25)


# Updating layout for title and labels
fig.update_layout(
    title_font_size=24,  
    legend_font_size=20,  
    legend_title_font_size=18 
)

# Chart Display
fig.show()

In [18]:
# Creating a table for building a map
df_country = wine_aroma_df[['Country']].groupby('Country').size().reset_index(name='Count')
df_country['Country'] = df_country['Country'].replace('USA', 'United States of America') # Replacing with the correct name
df_country['Country'] = df_country['Country'].replace('Czech', 'Czechia') # Replacing with the correct name
df_country

,Country,Count
0,Australia,55
1,Bulgaria,2
2,Chile,40
3,China,10
4,Croatia,10
5,Czechia,2
6,France,6
7,Hungary,3
8,Italy,106
9,New Zealand,16


In [ ]:
# Creating a horopleth card using Plotly Express
fig = px.choropleth(df_country, 
                    locations='Country', 
                    locationmode='country names', 
                    color='Count',  # We set the color depending on the number of samples
                    hover_name='Country', 
                    color_continuous_scale=px.colors.sequential.Teal,  # Choosing a color palette
                    title='Wine distribution map by country', width=800,  height=650
                    
                    )

fig.update_layout(
    title_font_size=24,  
    legend_font_size=20,  
    legend_title_font_size=18  
)

# Map Display
fig.show()

**Output:** Most of the wines in the dataset are from Spain, Italy, Australia and Chile, as these are the most popular wine regions, the rest of the wines are mainly from Europe and America, as these countries have the most developed winemaking.

### 2.2. Grape sort
Let's estimate how many grape varieties are represented in the data

In [ ]:
# Creating a pie chart
fig = px.pie(wine_aroma_df, 
             names='Grape sort', 
             color_discrete_sequence = px.colors.sequential.Teal_r,
             labels={'Grape sort': ''}, 
             width=800,  height=650  # Removing the percentage values
            )

# Setting labels
fig.update_traces(textposition='inside', textinfo='label+percent')

fig.update_layout(
    title_font_size=24,  
    legend_font_size=20,  
    legend_title_font_size=18 
)

# Chart Display
fig.show()

**Output** In the diagram, you can see that there is no clear predominance of one or more varieties, different varieties are represented, so this dataset is representative

### 2.3. Year
Let's also consider the distribution of data by year

In [ ]:
fig = px.histogram(wine_aroma_df, 
                   x='Year', 
                   color_discrete_sequence=['#3b738e'], 
                   width=800,  
                   height=650)

fig.update_layout(
    title_font_size=24,  
    legend_font_size=20,  
    legend_title_font_size=18,  
    xaxis=dict(
        title_font=dict(size=20),  
        tickfont=dict(size=20)  
    ),
    yaxis=dict(
        title_font=dict(size=20),  
        tickfont=dict(size=20) 
    )
)

# Chart Display
fig.show()

**Output:** As we can see on the histogram, the dataset mainly presents data for the last 10 years, this is due to the fact that only now they have begun to actively study this area and publish works that contain chemical structure

## 3. Analysis of the chemical composition of the presented wines
Let's consider which compounds are represented in the dataset and in what concentration

In [22]:
chem_df = wine_aroma_df.loc[:, 'Acetoine':'Dodecanoic acid'] # A dataframe that contains only chemical parameters

chem_df.describe()

,Acetoine,Acetaldehyde,Methanol,1-propanol,Ethyl lactate,Isobutanol,1-butanol,2-butanol,2-methyl-1-butanol,3-methyl-1-butanol,...,1-octanol,Hexanoic acid,Octanoic acid,Decanoic acid,Propanoic acid,butyric acid,2-methylpropanoic acid,2-methylbutyric acid,3-methylbutyric acid,Dodecanoic acid
count,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,...,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000,449.000000
mean,3.013686,9.370619,10.108518,6.276215,16.704722,15.561368,0.812189,0.067253,16.465266,48.323271,...,0.007184,1.407657,1.786861,0.858902,0.064838,0.229574,0.169252,1.564797,0.084764,0.001719
std,17.931572,33.164155,27.588826,15.327420,56.265246,34.157441,2.098714,0.481374,38.685584,83.351012,...,0.025686,1.738032,2.317287,3.147311,0.555783,0.461500,0.698909,12.690898,0.378875,0.012896
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.008600,0.040900,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000800,0.000000,0.000000,0.000000,...,0.000000,0.865000,0.848000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,4.000000,6.256000,21.590000,0.640000,0.000000,8.140000,83.575000,...,0.000000,2.122000,2.890000,0.550000,0.000000,0.135000,0.000000,0.000000,0.000000,0.000000
max,234.000000,347.000000,161.000000,182.000000,652.152000,425.000000,13.420000,8.093000,373.020000,359.000000,...,0.260000,7.776000,11.131000,30.600000,8.450000,2.060000,7.161000,122.000000,2.806000,0.155000


### 3.1. Comparison of averages

In [ ]:
# Creating a graph for the mean values
fig = px.bar(chem_df.describe().loc['mean', :], 
             color_discrete_sequence=px.colors.sequential.Teal_r, 
             labels={'value': 'Mean'})

# Adding a graph for maximum values (max)
# Using the same color palette
fig.add_bar(x=chem_df.columns, 
            y=chem_df.describe().loc['max', :], 
            marker_color=px.colors.sequential.Teal_r[-1], 
            name='Max')

# Displaying the graph
fig.show()

**Output:** It can be seen from the graph that there are several compounds whose average concentration in wine exceeds all others, these are 3-methyl-1-butanol, 2,3-butanediol, 2-phenylethanol, Isoamyl alcohol, the rest are in equal concentrations, however, several compounds are almost not found in the resulting wines, this is Pentyl acetate, Phenyl acetate, Ethyl caprelate, Geraniol, 1-octanol

### 3.2. Correlations
To understand which chemical components are most correlated with each other, let's build a correlation matrix

In [27]:
# Calculate the correlation matrix
corr_matrix = chem_df.corr()

# Creating a triangular correlation matrix, hiding the upper triangle
tril_corr_matrix = np.tril(corr_matrix)

# Visualize the correlation matrix using Pony Express
fig = px.imshow(tril_corr_matrix, x=list(corr_matrix.columns), y=list(corr_matrix.columns), color_continuous_scale = px.colors.sequential.Teal_r)

# Increasing the size of the drawing
fig.update_layout(width=1000, height=1000)
fig.show()

**Conclusion**: In this picture, you can see that the strongest correlations are observed between
- Acetoine and 2,3-butanediol
- Ethyl octanoate and Ethyl decactanoate
- 1-hexanol and Ethel butyrate
- Ethyl octanoate and Ethyl hexanoate
- 1-hexnaol and Nerol


### 3.3. Columns with zeros
Let's check if there are columns that do not contain any values other than zeros

In [28]:
columns_with_only_zeros = chem_df.columns[chem_df.eq(0).all()]

if len(columns_with_only_zeros) > 0:
    print("Columns with only zero values:", columns_with_only_zeros)
else:
    print("All columns contain not only null values.")

All columns contain not only null values.


## 4. Aromas of wine
Let's consider separately the pillars describing the flavors of wine

In [29]:
aroma_df = wine_aroma_df.loc[:, 'Herbs and spices':]
aroma_df

,Herbs and spices,Tobacco/Smoke,Wood,Berries,Citrus,Fruits,Nuts,Coffee,Chocolate/Cacao,Flowers
0,0,1,0,1,0,0,0,0,1,0
1,1,1,0,1,0,0,0,0,1,0
2,0,0,0,1,0,0,0,0,0,0
3,1,1,0,1,0,1,0,1,1,0
4,1,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
444,0,0,0,1,0,1,0,0,0,0
445,0,0,0,1,0,0,0,0,0,0
446,1,0,0,1,0,0,0,0,0,0
447,0,1,0,1,0,0,1,0,1,1


### 4.1. Frequency of aromas
Let's estimate how often certain flavors are found in the table and build a diagram

In [30]:
# Calculate the sum of the values in the specified columns
sums = aroma_df[['Herbs and spices', 'Tobacco/Smoke', 'Wood', 'Berries', 'Citrus', 'Fruits', 'Nuts', 'Coffee', 'Chocolate/Cacao', 'Flowers']].sum()
sums_aroma = pd.DataFrame(sums, columns = ['Sum'])
sums_aroma = sums_aroma.reset_index()
sums_aroma

,index,Sum
0,Herbs and spices,278
1,Tobacco/Smoke,47
2,Wood,74
3,Berries,227
4,Citrus,108
5,Fruits,303
6,Nuts,34
7,Coffee,23
8,Chocolate/Cacao,47
9,Flowers,151


In [ ]:
# Creating a pie chart
fig = px.pie(sums_aroma, values = 'Sum',  
             names='index', 
             color_discrete_sequence=px.colors.sequential.Teal_r,
             labels={'Grape sort': ''}, 
             width=800,  height=650  # Removing the percentage values
            )

# Setting labels
fig.update_traces(textposition='inside', textinfo='label+percent', textfont_size=25)

fig.update_layout(
    title_font_size=24,  
    legend_font_size=20,  
    legend_title_font_size=18  
)

# Chart Display
fig.show()

**Output:** As you can see, most of the samples contain notes of fruits, herbs, flowers, berries, more unique smells such as coffee, nuts are rare, therefore, they may cause problems when predicting

### 4.2. The relationship of flavor and chemical composition
Let's build correlation matrices that reflect the relationship between the aroma of wine and its chemical composition to make sure that our dataset corresponds to reality
#### 4.2.1. Pearson Correlation Coefficient

In [ ]:
# Creating a table containing the chemical composition and aromatic characteristics
aroma_chem_df = wine_aroma_df.loc[:, 'Acetoine':]
aroma_chem_df = aroma_chem_df.drop(['DOI', 'Descriptors'], axis = 1)

# Calculate the correlation between chemical composition and aromatic characteristics
correlation_matrix = aroma_chem_df.corr().tail(10).loc[:, 'Acetoine' : 'Dodecanoic acid']

# Building a heat map with rotated x-axis labels
fig = px.imshow(correlation_matrix, color_continuous_scale=px.colors.sequential.Teal)

fig.update_xaxes(tickangle=45)  

fig.update_layout(
    title_font_size=24,  
    legend_font_size=20,  
    legend_title_font_size=18  
)

fig.show()

#### 4.2.3. Spearman Correlation Coefficient

In [ ]:
# Creating a table containing the chemical composition and aromatic characteristics
aroma_chem_df = wine_aroma_df.loc[:, 'Acetoine':]
aroma_chem_df = aroma_chem_df.drop(['DOI', 'Descriptors'], axis = 1)

# Calculate the correlation between chemical composition and aromatic characteristics
correlation_matrix = aroma_chem_df.corr(method='spearman').tail(10).loc[:, 'Acetoine' : 'Dodecanoic acid']

# Building a heat map with rotated x-axis labels
fig = px.imshow(correlation_matrix, color_continuous_scale=px.colors.sequential.Teal)

fig.update_xaxes(tickangle=45)  

fig.show()

**Output:** In this way, we do not see strong correlations between a particular compound and flavor

### General conclusion:
- The dataset contains 449 wines collected from 78 scientific articles
- Wines are presented from 15 countries, dominated by Spain and Italy, as the most developed wine regions
- Various grape varieties and blends are also presented, which allows us to conclude that the dataset is diverse
- Each wine is characterized by 49 chemical compounds that affect the flavor of the wine
- There is a certain imbalance in the aromas of various wines, most of the wines contain odors such as fruits, herbs and berries, fewer nuts and coffee, which may complicate the prediction